In [3]:
import pandas as pd
import numpy as np
import gensim
import csv

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec                                   #pip install gensim
from gensim.models.word2vec import Text8Corpus
import gensim.downloader as api
from glove import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

2021-12-15 19:10:59.627283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 19:10:59.627307: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def create_corpus(train_set):
    
    corpus=[]
    for tweet in train_set:
        # words=[word.lower() for word in word_tokenize(tweet) if(len(tweet)>1 and word.isalpha())]
        corpus.append(tweet)
        corpus = [element for element in corpus if len(element)>0]
        
    return corpus

In [5]:
def create_pretrained() : 
    embedding_dict={}
    with open('../Resources/twitter_dict/glove.twitter.27B.200d.txt','rb') as f:
        for line in f:
            values=line.split()
            vectors=np.asarray(values[1:],'float32')
            embedding_dict[values[0].decode()]=vectors
    f.close()
    
    return embedding_dict

In [6]:
def create_glove_emb(train_set) :
    model = Corpus()
    train_splitted = [tweet.split() for tweet in train_set]
    model.fit(train_splitted, window = 5)
    
    glove = Glove(no_components=200, learning_rate=0.05)
    glove.fit(model.matrix, epochs=50)
    glove.add_dictionary(model.dictionary)
    glove.save('glove.model')
    
    embedding_dict = {}
    for w, id_ in glove.dictionary.items():
        embedding_dict[w] = np.array(glove.word_vectors[id_])

    return embedding_dict

In [7]:
def load_embedding_dict(train_set, use_pretrained=True):
    
    if (use_pretrained) : 
        embedding_dict = create_pretrained()
    else : 
        embedding_dict = create_glove_emb(train_set)
    
    return embedding_dict

In [13]:
def create_sequence(corpus, size_pos, size_neg, max_length=None) : 

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    word_index = tokenizer.word_index
    nb_words = len(word_index)
    sequences = tokenizer.texts_to_sequences(corpus)
    
    tweet_pad = pad_sequences(sequences,maxlen=max_length,truncating='post',padding='post')
    print(len(sequences), len(corpus), len(tweet_pad))
    y = np.array(int(size_pos/2) * [0] + int(size_neg/2) * [1])
 
    indices = np.arange(tweet_pad.shape[0])
    np.random.shuffle(indices)
    tweet_pad = tweet_pad[indices]
    y = y[indices]

    return tweet_pad , word_index , nb_words, y 

In [14]:
def we_glove(train_set,size_pos, size_neg, use_pretrained = True, max_length = None):
    #corpus = create_corpus(train_set)
    
    sequence, word_index, nb_words, y = create_sequence(train_set,size_pos, size_neg, max_length)
    num_words=nb_words+1
    embedding_dict = load_embedding_dict(train_set, use_pretrained)
    glove_matrix = np.zeros((num_words, 200))

    for word,i in word_index.items():
        if i >= num_words:
            continue
        if use_pretrained : 
            emb_vec = embedding_dict.get(word)
        else : 
            emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            glove_matrix[i]=emb_vec
            
    return sequence, glove_matrix, embedding_dict, num_words, y 

In [15]:
PATH_TRAIN_NEG = '../Resources/preprocessing_neg1.txt'
PATH_TRAIN_POS = '../Resources/preprocessing_pos1.txt'

with open(PATH_TRAIN_POS) as f:
    train_pos = f.read().splitlines()
with open(PATH_TRAIN_NEG) as f:
    train_neg = f.read().splitlines()

train_set = train_pos + train_neg

In [16]:
sequence1, glove_matrix1, embedding_dict1, nb_words1, y1  = we_glove(train_set,len(train_pos), len(train_neg),False)

172295 172295


IndexError: index 156459 is out of bounds for axis 0 with size 86147

In [64]:
# define model
model = Sequential()
e = Embedding(nb_words1, 200, weights=[glove_matrix1], input_length=sequence1.shape[1], trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 968910, 200)       13847400  
                                                                 
 flatten_2 (Flatten)         (None, 193782000)         0         
                                                                 
 dense_2 (Dense)             (None, 1)                 193782001 
                                                                 
Total params: 207,629,401
Trainable params: 193,782,001
Non-trainable params: 13,847,400
_________________________________________________________________
None


In [65]:
print(sequence1.shape)
print(y1)


(2, 968910)
[1 0]


In [66]:
model.fit(sequence1, y1, epochs=50, verbose=0)

In [1]:
PATH_TRAIN_POS = '../Resources/test_data_processed.txt'
with open(PATH_TRAIN_NEG) as f:
    test_set = f.read().splitlines()


NameError: name 'PATH_TRAIN_NEG' is not defined

In [ ]:
test_sequence, glove_matrix_test, embedding_dict_test, nb_words_test, y_test = we_glove(train_set,False)

In [ ]:
print(test_sequence.shape)

(2, 968910)


In [ ]:
predictions = model.predict(test_sequence)

In [ ]:
print(predictions.shape)
print(predictions)

(2, 1)
[[0.]
 [1.]]


In [ ]:
with open('../Resources/glove_result.csv', 'w') as out:
    writer = csv.writer(out)
    writer.writerow(["Id", "Prediction"])
    for a, b in predictions:
        if b == 0:
            writer.writerow([a, -1])
        else:
            writer.writerow([a, 1])